## Community Health Equity Lab: New York COVID19 Response Analysis

In [1]:
import pandas as pd
import numpy as np
import warnings
from plotnine import *
#import bamboolib as bam

from sklearn.model_selection import train_test_split # simple TT split cv

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.options.display.max_columns = 300
pd.set_option('display.max_rows', 500)

## Data

In [2]:
#reading zip code data and joining borough name
ny = pd.read_csv('data/ny18_cleaned.csv')
ny.head()

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use
0,10065,984654,0,28109,24285,86.4,619,2.2,37,0.1,2666,9.5,0,0.0,154,0.5,348,1.2,12248,15861,1650,5.9,1000,3.6,916,3.3,356,1.3,1785,6.4,2671,9.5,3176,11.3,2558,9.1,1607,5.7,1570,5.6,1796,6.4,1460,5.2,1522,5.4,1388,4.9,1486,5.3,1159,4.1,954,3.4,1055,3.8,1894,6.8,70.2,14844,414,2.8,27635,22612,2247,5.0,2.7,4.0,2.7,6.2,10.6,8.8,14.4,9.8,35.8,127375,242978,19681,3045,3306,27963,1922,3.0,109,1683,3797,2267,275,13,27299,97.3,744,2.7,5365,9479,233,682,34.16,121,385,0.314286,121,385,0.314286,171,492,34.76,187,544,34.38,14,7625,7993,7993,6493,5519,745,745,402,6.647990,974,181
1,10069,249050,0,5085,3155,62.0,148,2.9,0,0.0,1558,30.6,0,0.0,0,0.0,224,4.4,2354,2731,368,7.2,416,8.2,274,5.4,95,1.9,235,4.6,427,8.4,594,11.7,639,12.6,508,10.0,374,7.4,378,7.4,235,4.6,197,3.9,61,1.2,87,1.7,109,2.1,0,0.0,88,1.7,182,3.6,74.7,2552,73,2.9,5025,3790,859,8.7,3.2,0.0,9.8,5.7,4.2,14.7,11.4,11.6,30.9,110625,225183,2922,331,1207,5085,522,7.3,25,88,517,662,27,0,4992,98.2,93,1.8,675,1877,40,105,38.10,24,57,0.421053,24,57,0.421053,29,72,40.28,32,81,39.51,9,4922,4922,4922,4093,3765,410,410,273,4.741397,328,138
2,10075,477137,0,21556,18396,85.3,677,3.1,225,1.0,1047,4.9,0,0.0,902,4.2,309,1.4,10096,11460,1041,4.8,1252,5.8,736,3.4,763,3.5,778,3.6,1400,6.5,2022,9.4,1643,7.6,1148,5.3,1453,6.7,1244,5.8,1118,5.2,1131,5.2,1561,7.2,1276,5.9,1169,5.4,570,2.6,1251,5.8,1689,8.0,68.1,11035,217,2.0,21381,18455,1508,1.9,0.8,4.8,6.4,5.0,12.8,10.6,17.9,9.6,30.3,137146,233358,13727,3315,2372,21155,1075,4.5,129,1401,3133,1765,236,0,20715,97.9,440,2.1,4933,6102,294,631,46.59,160,371,0.431267,160,371,0.431267,204,452,45.13,232,501,46.31,15,7573,7729,7729,6277,5678,788,788,412,7.253486,599,198
3,10128,1206191,0,59256,47167,79.6,2182,3.7,0,0.0,5844,9.9,0,0.0,1666,2.8,2397,4.0,25338,33918,3204,5.4,2138,3.6,1911,3.2,1691,2.9,2644,4.5,6825,11.5,7216,12.2,4349,7.3,3849,6.5,4103,6.9,3845,6.5,2989,5.0,3494,5.9,3461,5.8,2656,4.5,1980,3.3,1557,2.6,1344,2.3,4825,8.1,72.2,30691,1506,4.9,58419,48263,4921,4.5,2.2,6.5,3.6,5.6,12.4,11.2,14.6,10.0,29.4,114010,196844,39143,7054,6535,58980,3686,3.4,500,3513,8171,5192,530,3,57424,96.9,1820,3.1,10129,20562,460,1207,38.11,212,596,0.355705,212,596,0.355705,281,737,38.13,317,834,38.01,13,5936,6233,6233,4713,4177,514,514,337,6.464822,536,147
4,10280,297253,38409,9384,7360,78.4,184,2.0,0,0.0,1474,15.7,0,0.0,77,0.8,289,3.1,4477,4907,711,7.6,628,6.7,454,4.8,192,2.0,239,2.5,523,5.6,1374,14.6,945,10.1,1021,10.9,771,8.2,763,8.1,530,5.6,375,4.0,370,3.9,254,2.7,

In [3]:
boroughs_zip = pd.read_csv('data/bor_zip_codes.csv')
ny = pd.merge(ny,boroughs_zip[['GEOID10', 'borough']], left_on='GEOID_ZIP', right_on='GEOID10')
del(boroughs_zip)
ny.head()

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use,GEOID10,borough
0,10065,984654,0,28109,24285,86.4,619,2.2,37,0.1,2666,9.5,0,0.0,154,0.5,348,1.2,12248,15861,1650,5.9,1000,3.6,916,3.3,356,1.3,1785,6.4,2671,9.5,3176,11.3,2558,9.1,1607,5.7,1570,5.6,1796,6.4,1460,5.2,1522,5.4,1388,4.9,1486,5.3,1159,4.1,954,3.4,1055,3.8,1894,6.8,70.2,14844,414,2.8,27635,22612,2247,5.0,2.7,4.0,2.7,6.2,10.6,8.8,14.4,9.8,35.8,127375,242978,19681,3045,3306,27963,1922,3.0,109,1683,3797,2267,275,13,27299,97.3,744,2.7,5365,9479,233,682,34.16,121,385,0.314286,121,385,0.314286,171,492,34.76,187,544,34.38,14,7625,7993,7993,6493,5519,745,745,402,6.647990,974,181,10065,Manhattan
1,10069,249050,0,5085,3155,62.0,148,2.9,0,0.0,1558,30.6,0,0.0,0,0.0,224,4.4,2354,2731,368,7.2,416,8.2,274,5.4,95,1.9,235,4.6,427,8.4,594,11.7,639,12.6,508,10.0,374,7.4,378,7.4,235,4.6,197,3.9,61,1.2,87,1.7,109,2.1,0,0.0,88,1.7,182,3.6,74.7,2552,73,2.9,5025,3790,859,8.7,3.2,0.0,9.8,5.7,4.2,14.7,11.4,11.6,30.9,110625,225183,2922,331,1207,5085,522,7.3,25,88,517,662,27,0,4992,98.2,93,1.8,675,1877,40,105,38.10,24,57,0.421053,24,57,0.421053,29,72,40.28,32,81,39.51,9,4922,4922,4922,4093,3765,410,410,273,4.741397,328,138,10069,Manhattan
2,10075,477137,0,21556,18396,85.3,677,3.1,225,1.0,1047,4.9,0,0.0,902,4.2,309,1.4,10096,11460,1041,4.8,1252,5.8,736,3.4,763,3.5,778,3.6,1400,6.5,2022,9.4,1643,7.6,1148,5.3,1453,6.7,1244,5.8,1118,5.2,1131,5.2,1561,7.2,1276,5.9,1169,5.4,570,2.6,1251,5.8,1689,8.0,68.1,11035,217,2.0,21381,18455,1508,1.9,0.8,4.8,6.4,5.0,12.8,10.6,17.9,9.6,30.3,137146,233358,13727,3315,2372,21155,1075,4.5,129,1401,3133,1765,236,0,20715,97.9,440,2.1,4933,6102,294,631,46.59,160,371,0.431267,160,371,0.431267,204,452,45.13,232,501,46.31,15,7573,7729,7729,6277,5678,788,788,412,7.253486,599,198,10075,Manhattan
3,10128,1206191,0,59256,47167,79.6,2182,3.7,0,0.0,5844,9.9,0,0.0,1666,2.8,2397,4.0,25338,33918,3204,5.4,2138,3.6,1911,3.2,1691,2.9,2644,4.5,6825,11.5,7216,12.2,4349,7.3,3849,6.5,4103,6.9,3845,6.5,2989,5.0,3494,5.9,3461,5.8,2656,4.5,1980,3.3,1557,2.6,1344,2.3,4825,8.1,72.2,30691,1506,4.9,58419,48263,4921,4.5,2.2,6.5,3.6,5.6,12.4,11.2,14.6,10.0,29.4,114010,196844,39143,7054,6535,58980,3686,3.4,500,3513,8171,5192,530,3,57424,96.9,1820,3.1,10129,20562,460,1207,38.11,212,596,0.355705,212,596,0.355705,281,737,38.13,317,834,38.01,13,5936,6233,6233,4713,4177,514,514,337,6.464822,536,147,10128,Manhattan
4,10280,297253,38409,9384,7360,78.4,184,2.0,0,0.0,1474,15.7,0,0.0,77,0.8,289,3.1,4477,4907,711,7.6,628,6.7,454,4.8,192,2.0,239,2.5,523,5

In [4]:
gm = pd.read_csv('data/Google_Mobility_Report_Filtered.csv')
gm.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,New York,Bronx,2/15/2020,0,-8,-5,-3,0,1
1,US,United States,New York,Bronx,2/16/2020,-1,-4,5,-2,-2,0
2,US,United States,New York,Bronx,2/17/2020,3,-8,-11,-22,-38,10
3,US,United States,New York,Bronx,2/18/2020,-2,-6,-11,-7,-10,3
4,US,United States,New York,Bronx,2/19/2020,2,-6,0,-6,-9,2


In [5]:
# using an aggregate of drop in baseline from "normal" starting from Feb 15 - Apr 11
# figured an aggregate would be best way to deal with the abundance of data
gm_agg = gm.groupby(['sub_region_2']).agg({col: ['sum'] for col in ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']})
gm_agg.columns = ['_'.join(multi_index) for multi_index in gm_agg.columns.ravel()]
gm_agg = gm_agg.reset_index()
gm_agg

,sub_region_2,retail_and_recreation_percent_change_from_baseline_sum,grocery_and_pharmacy_percent_change_from_baseline_sum,parks_percent_change_from_baseline_sum,transit_stations_percent_change_from_baseline_sum,workplaces_percent_change_from_baseline_sum,residential_percent_change_from_baseline_sum
0,Bronx,-1155,-211,-758,-1366,-1437,611
1,Brooklyn,-1407,-249,-45,-1677,-1601,715
2,Manhattan,-2296,-1006,-1544,-2199,-1926,777
3,Queens,-1470,-315,500,-1986,-1650,778
4,Staten Island,-1239,-254,-229,-1643,-1407,660


In [6]:
ny = pd.merge(ny, gm_agg, left_on='borough', right_on='sub_region_2')
ny.sample(10).head(10)

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use,GEOID10,borough,sub_region_2,retail_and_recreation_percent_change_from_baseline_sum,grocery_and_pharmacy_percent_change_from_baseline_sum,parks_percent_change_from_baseline_sum,transit_stations_percent_change_from_baseline_sum,workplaces_percent_change_from_baseline_sum,residential_percent_change_from_baseline_sum
31,10030,722535,0,29882,5038,16.9,18842,63.1,21,0.1,617,2.1,123,0.4,3357,11.2,1884,6.3,13233,16649,2107,7.1,1377,4.6,1077,3.6,1434,4.8,2106,7.0,3476,11.6,2559,8.6,2815,9.4,2048,6.9,2199,7.4,2267,7.6,1414,4.7,1684,5.6,829,2.8,828,2.8,696,2.3,303,1.0,663,2.2,4519,15.1,64.1,12736,4269,33.5,29527,26394,1331,12.8,11.9,15.0,8.2,9.6,13.6,8.4,10.9,3.0,6.6,37232,72377,18698,3513,3579,29852,8777,23.3,1284,1457,2606,2646,499,127,27415,91.7,2467,8.3,1471,11265,323,526,61.41,106,204,0.519608,106,204,0.519608,164,289,56.75,194,340,57.06,9,4583,5405,5405,2982,2927,276,276,159,4.894893,55,86,10030,Manhattan,Manhattan,-2296,-1006,-1544,-2199,-1926,777
5,10282,177847,55133,5593,4153,74.3,61,1.1,0,0.0,1163,20.8,0,0.0,13,0.2,203,3.6,2707,2886,934,16.7,397,7.1,289,5.2,119,2.1,44,0.8,382,6.8,594,10.6,832,14.9,727,13.0,492,8.8,199,3.6,122,2.2,132,2.4,111,2.0,81,1.4,0,0.0,0,0.0,138,2.5,164,2.9,76.7,2231,25,1.1,5294,4457,485,3.3,1.9,3.2,1.3,1.5,2.3,3.5,12.7,1.8,68.5,0,330244,3467,595,950,5593,190,4.4,11,64,470,765,137,0,5517,98.6,76,1.4,192,2039,52,108,48.15,21,42,0.500000,21,42,0.500000,32,61,52.46,36,67,53.73,3,215,224,224,249,148,23,23,8,0.983361,101,0,10282,Manhattan,Manhattan,-2296,-1006,-1544,-2199,-1926,777
158,11208,9265807,0,98660,25549,25.9,53666,54.4,339,0.3,5621,5.7,62,0.1,11620,11.8,1803,1.8,45869,52791,8567,8.7,7657,7.8,6547,6.6,6782,6.9,7446,7.5,8260,8.4,7444,7.5,6696,6.8,6204,6.3,6315,6.4,6838,6.9,5823,5.9,5147,5.2,3202,3.2,2490,2.5,1358,1.4,929,0.9,955,1.0,8927,9.1,57.3,32181,9257,28.8,97152,92646,2407,16.0,6.1,10.8,10.9,13.5,15.1,11.1,10.0,4.0,2.6,40733,57398,45024,21615,16461,98268,25327,23.0,2670,3741,6561,10889,4398,923,92091,93.5,6443,6.5,8377,23804,1263,1959,64.47,350,561,0.623886,350,561,0.623886,608,923,65.87,753,1168,64.47,1,530,530,530,530,288,29,29,30,0.629608,242,19,11208,Queens,Queens,-1470,-315,500,-1986,-1650,778
128,11362,6554282,39269,18721,9308,49.7,176,0.9,0,0.0,8098,43.3,0,0.0,602,3.2,537,2.9,8786,9935,962,5.1,1024,5.5,635,3.4,769,4.1,797,4.3,988,5.3,858,4.6,1328,7.1,1111,5.9,1098,5.9,1453,7.8,1541,8.2,1322,7.1,1316,7.0,1155,6.2,598,3.2,680,3.6,1086,5.8,1885,10.1,57.9,7151,319,4.5,18566,17454,498,3.3,

In [ ]:
# ny2 = pd.read_spss('data/NYCZCTA_2018_5YR_Estimate_FINAL_05052020.sav')
# ny2 = ny2.sort_values(by=['ZCTA'], ascending=[None])
# ny2.head()

## Data Transformations

In [7]:
ny['per_minority'] =  ny['per_black'] + ny['per_native'] + ny['per_asian'] + ny['per_hawaiian'] + ny['per_other'] + ny['per_two_or'] 

In [8]:
conditions = [(ny['per_minority'] <= 50.0),
              (ny['per_minority'] > 50.0)]
choices = [0,1]

ny['minority-majority-50'] = np.select(conditions, choices, default=2)

In [9]:
conditions = [(ny['per_minority'] <= 60.0),
              (ny['per_minority'] > 60.0)]
choices = [0,1]

ny['minority-majority-60'] = np.select(conditions, choices, default=2)

In [10]:
conditions = [(ny['per_minority'] <= 70.0),
              (ny['per_minority'] > 70.0)]
choices = [0,1]

ny['minority-majority-70'] = np.select(conditions, choices, default=2)

In [11]:
old_columns = [x for x in ny.columns if x not in ['per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70']]
insert_index = old_columns.index('4_16_2020_positive')
ny = ny[old_columns[:insert_index] + ['per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70'] + old_columns[insert_index:]]

In [12]:
ny["pop_density"] = ny['tot_pop']/ny['ALAND10']

In [13]:
ny = ny[['GEOID_ZIP', 'ALAND10', 'AWATER10', 'tot_pop', 'white', 'per_white', 'black', 'per_black', 'native', 'per_native', 'asian', 'per_asian', 'nativehawaiian ', 'per_hawaiian', 'other', 'per_other', 'two_or_mor', 'per_two_or', 'male', 'female', 'under_5', 'per_under_5', '5_to_9', 'per_5_to_9', '10_to_14', 'per_10_to_14', '15_to_19', 'per_15_to_19', '20_to_24', 'per_20_to_24', '25_to_29', 'per_25_to_29', '30_to_34', 'per_30_to_34', '35_to_39', 'per_35_to_39', '40_to_44', 'per_40_to_44', '45_to_49', 'per_45_to_49', '50_to_54', 'per_50_to_54', '55_to_59', 'per_55_to_59', '60_to_64', 'per_60_to_64', '65_to_69', 'per_65_to_69', '70_to_74', 'per_70_to_74', '75_to_79', 'per_75_to_79', '80_to_84', 'per_80_to_89', '80_and_over', 'per_80_and_over', 'disabled', 'per_disabled', 'unemployment_rate', 'tot_households_snap', 'households_snap', 'per_households_snap', 'tot_pop_mobility', 'same_house', 'moved_within_1yr', 'less_10k', '10k_15k', '15k_25k', '25k_35k', '35k_50k', '50k_75k', '75k_100k', '100k_150k', '150k_200k', 'more_200k', 'med_income', 'mean_incom', 'speaks_only_english', 'naturalized', 'non_citizen', 'pop_pov', 'pop_below_', 'pcnt_pov', 'families_on_suplimental_income', 'families_on_social_security', '2_ppl_fam', '3_to_4_ppl', '5_to_6_ppl', 'GT_7_ppl_f', 'insured', 'per_insured', 'uninsured', 'per_uninsured', 'owner', 'rent', 'per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70'] + ['pop_density'] + ['4_16_2020_positive', '4_16_2020_tests', '4_16_2020_positive_rate', '4_2_2020_positive', '4_2_2020_tests', '4_2_2020_positive_rate', '4_3_2020_positive', '4_3_2020_tests', '4_3_2020_positive_rate', '4_7_2020_positive', '4_7_2020_tests', '4_7_2020_positive_rate', '4_8_2020_positive', '4_8_2020_tests', '4_8_2020_positive_rate', 'hospital_count', '3_26_bb_beds', '4_2_bb_beds', '4_7_bb_beds', 'licensed_beds', 'staffed_beds', 'ICU_beds', 'adult_icu_beds', 'pediatrics_icu_beds', 'bed_utilization_rate', 'potential_increase_bed_capacity', 'avg_ventilator_use']]

## Initial OLS Model

In [14]:
predictors = ["per_households_snap", "med_income", "pcnt_pov", "families_on_suplimental_income", "families_on_social_security", "per_uninsured", "minority-majority-60", "pop_density"]

X_train, X_test, y_train, y_test = train_test_split(ny[predictors], ny["4_16_2020_positive"], test_size=0.2)

In [15]:
model = LinearRegression()
model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)


print('training r2 is:', model.score(X_train, y_train)) #training R2
print('testing r2 is:', model.score(X_test, y_test)) #testing R2

print('\ntrain mse is: ', mean_squared_error(y_train,train_pred))
print('test mse is: ', mean_squared_error(y_test,test_pred))

training r2 is: 0.6703265922394026
testing r2 is: 0.7240225717965059

train mse is:  84327.21983888421
test mse is:  41949.13301713943


In [16]:
ny.reset_index(inplace=True)
test_copy = X_test.copy()
test_copy.reset_index(inplace=True)

test_copy = pd.merge(test_copy, ny[['index','tot_pop','4_16_2020_positive']], on='index', how='left')
test_copy['ols_preds'] = test_pred
print(test_copy.head(25))

    index  per_households_snap  med_income  pcnt_pov  \
0     176                 14.4       83240       7.3   
1     131                  9.7       66483      11.6   
2     108                 30.2       41710      20.1   
3       3                  4.9      114010       3.4   
4      80                 14.2       50832       8.5   
5      17                  3.2      126601       3.9   
6      46                 23.4       62558      25.1   
7      89                 20.1       60873      11.7   
8      25                  4.5      131668       3.4   
9     171                 14.0       89235       5.4   
10    142                  7.1       73333       8.0   
11     29                  1.4      125140       1.1   
12     42                  9.9      100441       3.9   
13    183                  7.5       97338       4.5   
14     54                  4.3       95064       5.0   
15    179                 15.3       62207       7.7   
16    192                  0.0           0      

In [17]:
coefficients = pd.DataFrame({"Coef":model.coef_,
              "Name": predictors})

coefficients = coefficients.append({"Coef": model.intercept_,
               "Name": "intercept"}, ignore_index = True)

print(coefficients)

         Coef                            Name
0    3.072529             per_households_snap
1    0.000091                      med_income
2   -4.171202                        pcnt_pov
3    0.157964  families_on_suplimental_income
4    0.155137     families_on_social_security
5   21.799213                   per_uninsured
6   45.371995            minority-majority-60
7 -435.655624                     pop_density
8 -119.633473                       intercept


In [18]:
mod = sm.OLS(y_train,X_train)
fit = mod.fit()

In [19]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     4_16_2020_positive   R-squared (uncentered):                   0.872
Model:                            OLS   Adj. R-squared (uncentered):              0.865
Method:                 Least Squares   F-statistic:                              125.1
Date:                Wed, 13 May 2020   Prob (F-statistic):                    1.22e-61
Time:                        21:02:42   Log-Likelihood:                         -1099.2
No. Observations:                 155   AIC:                                      2214.
Df Residuals:                     147   BIC:                                      2239.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
per_households_snap                2.2918      5.585      0.410      0.682      -8.745      13.328
med_income                        -0.0007      0.001     -1.232      0.220      -0.002       0.000
pcnt_pov                          -6.1774      7.585     -0.814      0.417     -21.168       8.813
families_on_suplimental_income     0.1708      0.056      3.040      0.003       0.060       0.282
families_on_social_security        0.1470      0.022      6.683      0.000       0.104       0.190
per_uninsured                     19.6860      7.122      2.764      0.006       5.612      33.760
minority-majority-60              44.5923     62.693      0.711      0.478     -79.305     168.489
pop_density                     -275.6943   2248.126     -0.123      0.903   -4718.516    4167.127
==============================================================================
Omnibus:                       19.438   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.561
Skew:                           0.457   Prob(JB):                     1.73e-11
Kurtosis:                       5.615   Cond. No.                     7.59e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.59e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
test_preds = fit.predict(X_test)

In [21]:
#print('testing r2 is:', model.score(test_preds, y_test)) #testing R2

ValueError: Expected 2D array, got 1D array instead:
array=[ 5.15111327e+02  6.39256626e+02  9.98079953e+02  5.58882324e+02
  7.34732117e+02  1.05841308e+02  3.48761548e+02  1.28743311e+03
  5.40401236e+02  5.08129055e+02  5.00181938e+02  3.84646216e+02
  1.51292916e+02  6.58807970e+02  9.95721130e+02  9.57958280e+02
  2.92633208e+02 -1.91455257e+01  2.81453743e+02  2.06327266e+02
  3.56472711e+02  1.09042700e+03  5.37225333e+02  8.69689949e+02
  5.48743963e+02  1.46478604e+03  1.12108822e+03  1.26989259e+03
  1.13439810e+03 -9.20973084e+00  4.97606514e+02 -2.40505538e+01
  5.14109423e+02 -1.20170473e+01 -9.45419650e-02  5.51500500e+00
  3.91322163e+02  7.58769407e+02  1.09507111e+03].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.